In [4]:
"""
Created on Mon Mar 16 16:18:07 2020
Import settings
@author: Stamatis
"""

""" import libraries """

import copy
import itertools
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import numpy_financial as npf
import pandas as pd
import pickle
import random
import tensorflow as tf
import time
import sys
from collections import Counter, deque, OrderedDict
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM, GRU, Flatten, RepeatVector, TimeDistributed
from keras.models import load_model
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from tqdm import tqdm

""" file and datetime settings """
settings = dict()
file_path = '/Users/luisaweiss/Downloads/Data'
settings['filename_electricity'] = file_path + '/data_load_'
settings['filename_weather'] = file_path + '/data_meteorological'
settings['weather_predictors'] = ['Temperature', 'GHI', 'Wind Speed']
settings['start_date'] = '2002-01-01 01:00:00'
settings['end_date'] = '2002-12-31 23:00:00'
settings['random_transition_probabilities'] = False
settings['use_exponential_voll'] = False
settings['exponential_multiplier'] = 0.2
settings['include_ev_load'] = False
settings['superposed_scenario'] = True
settings['annual_demand_growth_rate_percent'] = 1
settings['num_outage_simulation_runs'] = 3
settings['increasing_storage_prices'] = []

""" constants """
DAYS_IN_YEAR = 365
HRS_IN_YEAR = 8760
SEC_IN_MIN = 60
SEC_IN_HOUR = 3600
ELECTRICITY_PRICE_PER_KWH = 0.1386
DIESEL_PRICE_PER_GALLON = 2.459
DIESEL_KWH_PER_GALLON = 35.8 * 3.785 * (1 / 3.6)  # (MJ/l) * (l/gal) * (KWh/MJ)
DIESEL_PRICE_PER_KWH = DIESEL_PRICE_PER_GALLON / DIESEL_KWH_PER_GALLON

""" initialization """
decision_periods, years_in_period, mc_transition_prob = 20, 1, 0.8
interest_rate, payments_per_year, loan_horizon = 0.02, 12, 10

""" define facilities, power plants and storage units lists """
facilities_list = ['hospital', 'outpatient', 'supermarket', 'hotel', 'office', 'school', 'restaurant', 'residential']
power_plants_list = ['solar', 'onshore wind', 'offshore wind', 'diesel']
storage_units_list = ['lithium-ion', 'lead acid', 'vanadium redox', 'flywheel', 'pumped hydro']

""" class for microgrid facilities """
class MGridFacility:
    def __init__(self,cnt,crit_load,voll,ev_profile,num_ev,lolp_constraint,ccp_constraint,kkt_multiplier):
        self.cnt = cnt
        self.crit_load = crit_load
        self.voll = voll
        self.ev_profile = ev_profile
        self.num_ev = num_ev
        self.lolp_constraint = lolp_constraint
        self.ccp_constraint = ccp_constraint
        self.kkt_multiplier = kkt_multiplier
        self.yearly_data = None
        self.total_data = None

""" class for power plants """
class MGridPowerPlant:
    def __init__(self,price_list,price_probs,price_state,life_list,life_probs,life_state,eff_list,eff_probs,eff_state,levels,decom_cost,annual_om_cost_rate,cap,loan_years,om_cost,rem_life,cur_eff):
        self.price_list = price_list
        self.price_probs = price_probs
        self.price_state = price_state
        self.life_list = life_list
        self.life_probs = life_probs
        self.life_state = life_state
        self.eff_list = eff_list
        self.eff_probs = eff_probs
        self.eff_state = eff_state
        self.levels = levels
        self.decom_cost = decom_cost
        self.annual_om_cost_rate = annual_om_cost_rate
        self.cap = cap
        self.loan_years = loan_years
        self.loan_payment = 0
        self.rem_loan = 0
        self.om_cost = om_cost
        self.rem_life = rem_life
        self.cur_eff = cur_eff
        self.prod = None
        self.power = None

    def install_new(self, new_cap):
        self.cap = new_cap
        self.loan_payment = - payments_per_year * np.pmt(interest_rate / payments_per_year, self.loan_years * payments_per_year, self.price_list[self.price_state]) * new_cap
        self.rem_loan = self.loan_payment * self.loan_years
        self.om_cost = years_in_period * self.annual_om_cost_rate * self.price_list[self.price_state] * new_cap
        self.rem_life = self.life_list[self.life_state] if new_cap != 0 else 0
        self.cur_eff = self.eff_list[self.eff_state] if new_cap != 0 else 0

    def decrease_life(self):
        if self.rem_life <= years_in_period:
            self.cap = 0
            self.loan_payment = 0
            self.rem_loan = 0
        self.rem_life = max(self.rem_life-years_in_period, 0)

    def state_transition(self, feature):
        if random.uniform(0, 1) < getattr(self, feature + '_probs')[getattr(self, feature + '_state')]:
            setattr(self, feature + '_state', getattr(self, feature + '_state') + 1)

""" class for storage units """
class MGridStorageUnit:
    def __init__(self,price_list,price_probs,price_state,life_list,life_probs,life_state,eff_list,eff_probs,eff_state,dod_list,dod_probs,dod_state,initial_soc,levels,decom_cost,annual_om_cost_rate,annual_degr_rate,cap,loan_years,om_cost,rem_life,cur_eff,cur_dod):
        self.price_list = price_list
        self.price_probs = price_probs
        self.price_state = price_state
        self.life_list = life_list
        self.life_probs = life_probs
        self.life_state = life_state
        self.eff_list = eff_list
        self.eff_probs = eff_probs
        self.eff_state = eff_state
        self.dod_list = dod_list
        self.dod_probs = dod_probs
        self.dod_state = dod_state
        self.initial_soc = initial_soc
        self.levels = levels
        self.decom_cost = decom_cost
        self.annual_om_cost_rate = annual_om_cost_rate
        self.annual_degr_rate = annual_degr_rate
        self.cap = cap
        self.loan_years = loan_years
        self.loan_payment = 0
        self.rem_loan = 0
        self.om_cost = om_cost
        self.rem_life = rem_life
        self.cur_eff = cur_eff
        self.cur_dod = cur_dod

    def install_new(self, new_cap):
        self.cap = new_cap
        self.loan_payment = - payments_per_year * np.pmt(interest_rate / payments_per_year, self.loan_years * payments_per_year, self.price_list[self.price_state]) * new_cap
        self.rem_loan = self.loan_payment * self.loan_years
        self.om_cost = years_in_period * self.annual_om_cost_rate * self.price_list[self.price_state] * new_cap
        self.rem_life = self.life_list[self.life_state] if new_cap != 0 else 0
        self.cur_eff = self.eff_list[self.eff_state] if new_cap != 0 else 0
        self.cur_dod = self.dod_list[self.dod_state] if new_cap != 0 else 0

    def decrease_life(self):
        if self.rem_life <= years_in_period:
            self.cap = 0
            self.loan_payment = 0
            self.rem_loan = 0
        self.rem_life = max(self.rem_life-years_in_period, 0)
        self.cap = self.cap * (1 - years_in_period * self.annual_degr_rate)

    def state_transition(self, feature):
        if random.uniform(0, 1) < getattr(self, feature + '_probs')[getattr(self, feature + '_state')]:
            setattr(self, feature + '_state', getattr(self, feature + '_state') + 1)

""" microgrid facilities """
fcnt, fcrit_load, fvoll, fev_profile, fnum_ev, flolp_constraint, fccp_constraint, fkkt_multiplier = OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict()

""" hospital data """
key = 'hospital'
fcnt[key] = 2
fcrit_load[key] = 0.8
fvoll[key] = 25
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [0] * 24
fnum_ev[key] = 0
flolp_constraint[key] = 0.2
fccp_constraint[key] = 0.8
fkkt_multiplier[key] = 10**6

""" outpatient data """
key = 'outpatient'
fcnt[key] = 2
fcrit_load[key] = 0.8
fvoll[key] = 19
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [0] * 24
fnum_ev[key] = 0
flolp_constraint[key] = 0.2
fccp_constraint[key] = 0.8
fkkt_multiplier[key] = 3*10**5

""" supermarket data """
key = 'supermarket'
fcnt[key] = 3
fcrit_load[key] = 0.6
fvoll[key] = 10
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [0] * 24
fnum_ev[key] = 0
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" hotel data """
key = 'hotel'
fcnt[key] = 3
fcrit_load[key] = 0.5
fvoll[key] = 9
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [x / 2 for x in [9, 9, 9, 9, 9, 9, 6.67, 4.33, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4.33, 6.67, 9, 9]]  # data coming from Prof. Jafari's research work on charging profiles for various facilities (Building 2)
fnum_ev[key] = 20
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" office data """
key = 'office'
fcnt[key] = 5
fcrit_load[key] = 0.5
fvoll[key] = 8
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [x / 2 for x in [2, 2, 2, 2, 2, 2, 2, 2, 4.33, 6.67, 9, 9, 9, 9, 9, 9, 9, 6.67, 4.33, 2, 2, 2, 2, 2]]  # data coming from Prof. Jafari's research work on charging profiles for various facilities (Office)
fnum_ev[key] = 10
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" school data """
key = 'school'
fcnt[key] = 3
fcrit_load[key] = 0.4
fvoll[key] = 7
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [0] * 24
fnum_ev[key] = 0
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" restaturant data """
key = 'restaurant'
fcnt[key] = 7
fcrit_load[key] = 0.9
fvoll[key] = 6
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [0] * 24
fnum_ev[key] = 0
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" residential data """
key = 'residential'
fcnt[key] = 300
fcrit_load[key] = 0.3
fvoll[key] = 5
assert fvoll[key] == min(fvoll.values()), "Facilities should be initialized in a voll-sorted fashion!"
fev_profile[key] = [x / 2 for x in [8, 8, 8, 8, 8, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 6]]  # data coming from Prof. Jafari's research work on charging profiles for various facilities (Apartment)
fnum_ev[key] = 2
flolp_constraint[key] = 0.5
fccp_constraint[key] = 0.5
fkkt_multiplier[key] = 10**5

""" power plants """
pprice_list, pprice_probs, pprice_state, plife_list, plife_probs, plife_state, peff_list, peff_probs, peff_state, plevels, pdecom_cost, pannual_om_cost_rate, pcap, ploan_years, pom_cost, prem_life, pcur_eff = OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict()

""" solar data """
key = 'solar'
pprice_list[key] = [357.64675326, 349.64214247, 341.63753168, 333.63292089, 325.6283101, 317.62369932, 309.61908853, 301.61447774, 293.60986695, 285.60525616, 277.60064538, 269.59603459, 261.5914238, 253.58681301, 245.58220222, 237.57759144, 229.57298065, 221.56836986, 213.56375907, 205.55914829]     # remember that these are costs per solar panel, NOT cost per kw
pprice_list[key] = pprice_list[key][0::years_in_period]
pprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
pprice_state[key] = 0
plife_list[key] = [33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42]
plife_list[key] = plife_list[key][0::years_in_period]
plife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
plife_state[key] = 0
peff_list[key] = [0.17068049999999999, 0.18056824999999999, 0.19045600000000001, 0.20034375000000001, 0.21023150000000002, 0.22011925000000002, 0.23000700000000002, 0.23989475000000002, 0.24978250000000002, 0.25967025000000005, 0.269558, 0.27944575000000005, 0.2893335, 0.29922125000000005, 0.3091090000000001, 0.31899675000000005, 0.3288845000000001, 0.3387722500000001, 0.3486600000000001, 0.35854775000000005]
peff_list[key] = peff_list[key][0::years_in_period]
peff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
peff_state[key] = 0
plevels[key] = [2_000, 4_000, 6_000, 8_000, 10_000, 12_000, 14_000, 16_000, 18_000, 20_000]
pdecom_cost[key] = 17.225
pannual_om_cost_rate[key] = 0.0063
pcap[key] = 0
ploan_years[key] = loan_horizon
pom_cost[key] = years_in_period * pannual_om_cost_rate[key] * pprice_list[key][pprice_state[key]] * pcap[key]
prem_life[key] = plife_list[key][plife_state[key]] if pcap[key] > 0 else 0
pcur_eff[key] = peff_list[key][peff_state[key]] if pcap[key] > 0 else 0

""" onshore wind data """
key = 'onshore wind'
pprice_list[key] = [167023.22820896, 163479.42728742, 159935.62636588, 156391.82544434, 152848.02452279, 149304.22360125, 145760.42267971, 142216.62175816, 138672.82083662, 135129.01991508, 131585.21899353, 128041.41807199, 124497.61715045, 120953.81622891, 117410.01530736, 113866.21438582, 110322.41346428, 106778.61254273, 103234.81162119 , 99691.01069965]     # remember that these are costs per wind turbine, NOT cost per kw
pprice_list[key] = pprice_list[key][0::years_in_period]
pprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
pprice_state[key] = 0
plife_list[key] = [17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26]
plife_list[key] = plife_list[key][0::years_in_period]
plife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
plife_state[key] = 0
peff_list[key] = [0.48, 0.48, 0.48, 0.48, 0.49, 0.49, 0.49, 0.49, 0.49, 0.50, 0.50, 0.50, 0.50, 0.51, 0.51, 0.51, 0.51, 0.52, 0.52, 0.52, 0.52]
peff_list[key] = peff_list[key][0::years_in_period]
assert all([x <= 0.593 for x in peff_list[key]]), "Betz's law"
peff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
peff_state[key] = 0
plevels[key] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pdecom_cost[key] = 5737.940464592778
pannual_om_cost_rate[key] = 0.007
pcap[key] = 0
ploan_years[key] = loan_horizon
pom_cost[key] = years_in_period * pannual_om_cost_rate[key] * pprice_list[key][pprice_state[key]] * pcap[key]
prem_life[key] = plife_list[key][plife_state[key]] if pcap[key] > 0 else 0
pcur_eff[key] = peff_list[key][peff_state[key]] if pcap[key] > 0 else 0

""" offshore wind data """
key = 'offshore wind'
pprice_list[key] = [736444.55558858, 729724.32356109, 723004.09153359, 716283.8595061, 709563.6274786, 702843.39545111, 696123.16342361, 689402.93139612, 682682.69936862, 675962.46734113, 669242.23531364, 662522.00328614, 655801.77125865, 649081.53923115, 642361.30720366, 635641.07517616, 628920.84314867, 622200.61112117, 615480.37909368, 608760.14706618]     # remember that these are costs per wind turbine, NOT cost per kw
pprice_list[key] = pprice_list[key][0::years_in_period]
pprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
pprice_state[key] = 0
plife_list[key] = [17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26]
plife_list[key] = plife_list[key][0::years_in_period]
plife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
plife_state[key] = 0
peff_list[key] = [0.48, 0.48, 0.48, 0.48, 0.49, 0.49, 0.49, 0.49, 0.49, 0.50, 0.50, 0.50, 0.50, 0.51, 0.51, 0.51, 0.51, 0.52, 0.52, 0.52, 0.52]
peff_list[key] = peff_list[key][0::years_in_period]
assert all([x <= 0.593 for x in peff_list[key]]), "Betz's law"
peff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
peff_state[key] = 0
plevels[key] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pdecom_cost[key] = 36385.233540973386
pannual_om_cost_rate[key] = 0.014
pcap[key] = 0
ploan_years[key] = loan_horizon
pom_cost[key] = years_in_period * pannual_om_cost_rate[key] * pprice_list[key][pprice_state[key]] * pcap[key]
prem_life[key] = plife_list[key][plife_state[key]] if pcap[key] > 0 else 0
pcur_eff[key] = peff_list[key][peff_state[key]] if pcap[key] > 0 else 0

""" diesel data """
key = 'diesel'
pprice_list[key] = [800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800]
pprice_list[key] = pprice_list[key][0::years_in_period]
pprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
pprice_state[key] = 0
plife_list[key] = [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
plife_list[key] = plife_list[key][0::years_in_period]
plife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
plife_state[key] = 0
peff_list[key] = [0.39, 0.40, 0.41, 0.42, 0.43, 0.44, 0.44, 0.44, 0.44, 0.44, 0.45, 0.45, 0.45, 0.45, 0.45, 0.46, 0.46, 0.46, 0.46, 0.46]
peff_list[key] = peff_list[key][0::years_in_period]
peff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
peff_state[key] = 0
plevels[key] = [100, 400, 700, 1_000, 1_300, 1_600, 1_900, 2_200, 2_500, 2_800]
pdecom_cost[key] = 31
pannual_om_cost_rate[key] = 0.04375
pcap[key] = 0
ploan_years[key] = loan_horizon
pom_cost[key] = years_in_period * pannual_om_cost_rate[key] * pprice_list[key][pprice_state[key]] * pcap[key]
prem_life[key] = plife_list[key][plife_state[key]] if pcap[key] > 0 else 0
pcur_eff[key] = peff_list[key][peff_state[key]] if pcap[key] > 0 else 0

""" hydro data """
key = 'hydro'
pprice_list[key] = [1518.19907009, 1544.39814019, 1570.59721028, 1596.79628038, 1622.99535047, 1649.19442056, 1675.39349066, 1701.59256075, 1727.79163085, 1753.99070094, 1780.18977104, 1806.38884113, 1832.58791122, 1858.78698132, 1884.98605141, 1911.18512151, 1937.3841916, 1963.58326169, 1989.78233179, 2015.98140188]
pprice_list[key] = pprice_list[key][0::years_in_period]
pprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
pprice_state[key] = 0
plife_list[key] = [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40]
plife_list[key] = plife_list[key][0::years_in_period]
plife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
plife_state[key] = 0
peff_list[key] = [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]
peff_list[key] = peff_list[key][0::years_in_period]
peff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
peff_state[key] = 0
plevels[key] = [100, 400, 700, 1_000, 1_300, 1_600, 1_900, 2_200, 2_500, 2_800]
pdecom_cost[key] = 303.63981401800004
pannual_om_cost_rate[key] = 0.025
pcap[key] = 0
ploan_years[key] = loan_horizon
pom_cost[key] = years_in_period * pannual_om_cost_rate[key] * pprice_list[key][pprice_state[key]] * pcap[key]
prem_life[key] = plife_list[key][plife_state[key]] if pcap[key] > 0 else 0
pcur_eff[key] = peff_list[key][peff_state[key]] if pcap[key] > 0 else 0

""" storage units """
sprice_list, sprice_probs, sprice_state, slife_list, slife_probs, slife_state, seff_list, seff_probs, seff_state, sdod_list, sdod_probs, sdod_state, sinitial_soc, slevels, sdecom_cost, sannual_om_cost_rate, sannual_degr_rate, scap, sloan_years, som_cost, srem_life, scur_eff, scur_dod = OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict(), OrderedDict()

""" lithium-ion data """
key = 'lithium-ion'
sprice_list[key] = [470.0, 449.4736842105263, 428.9473684210526, 408.42105263157896, 387.89473684210526, 367.36842105263156, 346.8421052631579, 326.3157894736842, 305.7894736842105, 285.2631578947368, 264.7368421052631, 244.21052631578948, 223.68421052631578, 203.15789473684208, 182.63157894736844, 162.10526315789474, 141.57894736842104, 121.05263157894734, 100.52631578947364, 80.0]
if key in settings['increasing_storage_prices']:
    sprice_list[key] = [470.0, 478.94736842105266, 487.89473684210526, 496.8421052631579, 505.7894736842105, 514.7368421052631, 523.6842105263158, 532.6315789473684, 541.578947368421, 550.5263157894736, 559.4736842105264, 568.421052631579, 577.3684210526316, 586.3157894736842, 595.2631578947369, 604.2105263157895, 613.1578947368421, 622.1052631578948, 631.0526315789474, 640.0]
sprice_list[key] = sprice_list[key][0::years_in_period]
sprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sprice_state[key] = 0
slife_list[key] = [12, 13, 14, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23]
slife_list[key] = slife_list[key][0::years_in_period]
slife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
slife_state[key] = 0
seff_list[key] = [0.94, 0.94, 0.94, 0.94, 0.95, 0.95, 0.95, 0.95, 0.96, 0.96, 0.96, 0.96, 0.97, 0.97, 0.97, 0.97, 0.98, 0.98, 0.98, 0.98]
seff_list[key] = seff_list[key][0::years_in_period]
seff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
seff_state[key] = 0
sdod_list[key] = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90]
sdod_list[key] = sdod_list[key][0::years_in_period]
sdod_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sdod_state[key] = 0
sinitial_soc[key] = 1
slevels[key] = [1_500, 3_500, 5_500, 7_500, 9_500, 11_500, 13_500, 15_500, 17_500, 19_500]
sdecom_cost[key] = 220
sannual_om_cost_rate[key] = 0.025
sannual_degr_rate[key] = 0.0171
scap[key] = 0
sloan_years[key] = loan_horizon
som_cost[key] = years_in_period * sannual_om_cost_rate[key] * sprice_list[key][sprice_state[key]] * scap[key]
srem_life[key] = slife_list[key][slife_state[key]] if scap[key] > 0 else 0
scur_eff[key] = seff_list[key][seff_state[key]] if scap[key] > 0 else 0
scur_dod[key] = sdod_list[key][sdod_state[key]] if scap[key] > 0 else 0

""" lead acid data """
key = 'lead acid'
sprice_list[key] = [260.0, 248.94736842105263, 237.89473684210526, 226.8421052631579, 215.78947368421052, 204.73684210526315, 193.68421052631578, 182.63157894736844, 171.57894736842104, 160.5263157894737, 149.4736842105263, 138.42105263157896, 127.36842105263159, 116.31578947368422, 105.26315789473685, 94.21052631578948, 83.15789473684211, 72.10526315789474, 61.05263157894737, 50.0]
if key in settings['increasing_storage_prices']:
    sprice_list[key] = [260.0, 266.8421052631579, 273.6842105263158, 280.5263157894737, 287.36842105263156, 294.2105263157895, 301.0526315789474, 307.89473684210526, 314.7368421052632, 321.57894736842104, 328.42105263157896, 335.2631578947368, 342.10526315789474, 348.9473684210526, 355.7894736842105, 362.63157894736844, 369.4736842105263, 376.3157894736842, 383.1578947368421, 390.0]
sprice_list[key] = sprice_list[key][0::years_in_period]
sprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sprice_state[key] = 0
slife_list[key] = [9, 10, 11, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20]
slife_list[key] = slife_list[key][0::years_in_period]
slife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
slife_state[key] = 0
seff_list[key] = [0.80, 0.80, 0.80, 0.80, 0.81, 0.81, 0.81, 0.81, 0.82, 0.82, 0.82, 0.82, 0.83, 0.83, 0.83, 0.83, 0.84, 0.84, 0.84, 0.84]
seff_list[key] = seff_list[key][0::years_in_period]
seff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
seff_state[key] = 0
sdod_list[key] = [0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55]
sdod_list[key] = sdod_list[key][0::years_in_period]
sdod_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sdod_state[key] = 0
sinitial_soc[key] = 1
slevels[key] = [1_500, 3_500, 5_500, 7_500, 9_500, 11_500, 13_500, 15_500, 17_500, 19_500]
sdecom_cost[key] = 88
sannual_om_cost_rate[key] = 0.025
sannual_degr_rate[key] = 0.0171
scap[key] = 0
sloan_years[key] = loan_horizon
som_cost[key] = years_in_period * sannual_om_cost_rate[key] * sprice_list[key][sprice_state[key]] * scap[key]
srem_life[key] = slife_list[key][slife_state[key]] if scap[key] > 0 else 0
scur_eff[key] = seff_list[key][seff_state[key]] if scap[key] > 0 else 0
scur_dod[key] = sdod_list[key][sdod_state[key]] if scap[key] > 0 else 0

""" vanadium redox """
key = 'vanadium redox'
sprice_list[key] = [400.0, 383.1578947368421, 366.3157894736842, 349.4736842105263, 332.63157894736844, 315.7894736842105, 298.9473684210526, 282.10526315789474, 265.2631578947369, 248.42105263157896, 231.57894736842107, 214.73684210526318, 197.89473684210526, 181.05263157894737, 164.21052631578948, 147.3684210526316, 130.5263157894737, 113.68421052631578, 96.84210526315792, 80.0]
if key in settings['increasing_storage_prices']:
    sprice_list[key] = [400.0, 407.89473684210526, 415.7894736842105, 423.6842105263158, 431.57894736842104, 439.4736842105263, 447.36842105263156, 455.2631578947368, 463.1578947368421, 471.0526315789474, 478.9473684210526, 486.8421052631579, 494.7368421052631, 502.63157894736844, 510.5263157894737, 518.421052631579, 526.3157894736842, 534.2105263157895, 542.1052631578948, 550.0]
sprice_list[key] = sprice_list[key][0::years_in_period]
sprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sprice_state[key] = 0
slife_list[key] = [13, 14, 15, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24]
slife_list[key] = slife_list[key][0::years_in_period]
slife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
slife_state[key] = 0
seff_list[key] = [0.70, 0.71, 0.72, 0.73, 0.74, 0.74, 0.75, 0.75, 0.76, 0.76, 0.77, 0.77, 0.78, 0.78, 0.79, 0.79, 0.80, 0.80, 0.81, 0.81]
seff_list[key] = seff_list[key][0::years_in_period]
seff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
seff_state[key] = 0
sdod_list[key] = [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
sdod_list[key] = sdod_list[key][0::years_in_period]
sdod_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sdod_state[key] = 0
sinitial_soc[key] = 1
slevels[key] = [1_500, 3_500, 5_500, 7_500, 9_500, 11_500, 13_500, 15_500, 17_500, 19_500]
sdecom_cost[key] = 300
sannual_om_cost_rate[key] = 0.025
sannual_degr_rate[key] = 0.0171
scap[key] = 0
sloan_years[key] = loan_horizon
som_cost[key] = years_in_period * sannual_om_cost_rate[key] * sprice_list[key][sprice_state[key]] * scap[key]
srem_life[key] = slife_list[key][slife_state[key]] if scap[key] > 0 else 0
scur_eff[key] = seff_list[key][seff_state[key]] if scap[key] > 0 else 0
scur_dod[key] = sdod_list[key][sdod_state[key]] if scap[key] > 0 else 0

""" flywheel """
key = 'flywheel'
sprice_list[key] = [3100.0, 2989.4736842105262, 2878.9473684210525, 2768.421052631579, 2657.8947368421054, 2547.3684210526317, 2436.842105263158, 2326.315789473684, 2215.7894736842104, 2105.2631578947367, 1994.7368421052631, 1884.2105263157894, 1773.6842105263158, 1663.157894736842, 1552.6315789473683, 1442.1052631578948, 1331.578947368421, 1221.0526315789473, 1110.5263157894738, 1000.0]
if key in settings['increasing_storage_prices']:
    sprice_list[key] = [3100.0, 3121.0526315789475, 3142.1052631578946, 3163.157894736842, 3184.2105263157896, 3205.2631578947367, 3226.315789473684, 3247.3684210526317, 3268.421052631579, 3289.4736842105262, 3310.5263157894738, 3331.5789473684213, 3352.6315789473683, 3373.684210526316, 3394.7368421052633, 3415.7894736842104, 3436.842105263158, 3457.8947368421054, 3478.9473684210525, 3500.0]
sprice_list[key] = sprice_list[key][0::years_in_period]
sprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sprice_state[key] = 0
slife_list[key] = [20, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30]
slife_list[key] = slife_list[key][0::years_in_period]
slife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
slife_state[key] = 0
seff_list[key] = [0.83, 0.83, 0.84, 0.84, 0.85, 0.85, 0.86, 0.86, 0.87, 0.87, 0.88, 0.88, 0.89, 0.89, 0.90, 0.90, 0.91, 0.91, 0.92, 0.92]
seff_list[key] = seff_list[key][0::years_in_period]
seff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
seff_state[key] = 0
sdod_list[key] = [0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86, 0.86]
sdod_list[key] = sdod_list[key][0::years_in_period]
sdod_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sdod_state[key] = 0
sinitial_soc[key] = 1
slevels[key] = [1_500, 3_500, 5_500, 7_500, 9_500, 11_500, 13_500, 15_500, 17_500, 19_500]
sdecom_cost[key] = 50
sannual_om_cost_rate[key] = 0.025
sannual_degr_rate[key] = 0
scap[key] = 0
sloan_years[key] = loan_horizon
som_cost[key] = years_in_period * sannual_om_cost_rate[key] * sprice_list[key][sprice_state[key]] * scap[key]
srem_life[key] = slife_list[key][slife_state[key]] if scap[key] > 0 else 0
scur_eff[key] = seff_list[key][seff_state[key]] if scap[key] > 0 else 0
scur_dod[key] = sdod_list[key][sdod_state[key]] if scap[key] > 0 else 0

""" pumped hydro """
key = 'pumped hydro'
sprice_list[key] = [1000.0, 989.4736842105264, 978.9473684210526, 968.421052631579, 957.8947368421052, 947.3684210526316, 936.8421052631579, 926.3157894736842, 915.7894736842105, 905.2631578947369, 894.7368421052631, 884.2105263157895, 873.6842105263158, 863.1578947368421, 852.6315789473684, 842.1052631578948, 831.578947368421, 821.0526315789473, 810.5263157894736, 800.0]
if key in settings['increasing_storage_prices']:
    sprice_list[key] = [1000.0, 1031.578947368421, 1063.157894736842, 1094.7368421052631, 1126.3157894736842, 1157.8947368421052, 1189.4736842105262, 1221.0526315789473, 1252.6315789473683, 1284.2105263157896, 1315.7894736842104, 1347.3684210526317, 1378.9473684210525, 1410.5263157894738, 1442.1052631578948, 1473.6842105263158, 1505.2631578947369, 1536.842105263158, 1568.421052631579, 1600.0]
sprice_list[key] = sprice_list[key][0::years_in_period]
sprice_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sprice_state[key] = 0
slife_list[key] = [60, 60, 60, 60, 60, 61, 61, 61, 61, 61, 62, 62, 62, 62, 62, 63, 63, 63, 63, 63]
slife_list[key] = slife_list[key][0::years_in_period]
slife_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
slife_state[key] = 0
seff_list[key] = [0.80, 0.80, 0.80, 0.80, 0.80, 0.81, 0.81, 0.81, 0.81, 0.81, 0.82, 0.82, 0.82, 0.82, 0.82, 0.83, 0.83, 0.83, 0.83, 0.83]
seff_list[key] = seff_list[key][0::years_in_period]
seff_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
seff_state[key] = 0
sdod_list[key] = [0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90, 0.90]
sdod_list[key] = sdod_list[key][0::years_in_period]
sdod_probs[key] = (decision_periods - 1) * [mc_transition_prob] + [0]
sdod_state[key] = 0
sinitial_soc[key] = 1
slevels[key] = [1_500, 3_500, 5_500, 7_500, 9_500, 11_500, 13_500, 15_500, 17_500, 19_500]
sdecom_cost[key] = 200
sannual_om_cost_rate[key] = 0.025
sannual_degr_rate[key] = 0
scap[key] = 0
sloan_years[key] = loan_horizon
som_cost[key] = years_in_period * sannual_om_cost_rate[key] * sprice_list[key][sprice_state[key]] * scap[key]
srem_life[key] = slife_list[key][slife_state[key]] if scap[key] > 0 else 0
scur_eff[key] = seff_list[key][seff_state[key]] if scap[key] > 0 else 0
scur_dod[key] = sdod_list[key][sdod_state[key]] if scap[key] > 0 else 0

""" initialize the facilities, power_plants and storage_units OrderedDict """
facilities, power_plants, storage_units = OrderedDict(), OrderedDict(), OrderedDict()
for facility in facilities_list:
    facilities[facility] = MGridFacility(fcnt[facility], fcrit_load[facility], fvoll[facility], fev_profile[facility], fnum_ev[facility], flolp_constraint[facility], fccp_constraint[facility], fkkt_multiplier[facility])
for power_plant in power_plants_list:
    power_plants[power_plant] = MGridPowerPlant(pprice_list[power_plant], pprice_probs[power_plant], pprice_state[power_plant], plife_list[power_plant], plife_probs[power_plant], plife_state[power_plant], peff_list[power_plant], peff_probs[power_plant], peff_state[power_plant], plevels[power_plant], pdecom_cost[power_plant], pannual_om_cost_rate[power_plant], pcap[power_plant], ploan_years[power_plant], pom_cost[power_plant], prem_life[power_plant], pcur_eff[power_plant])
for storage_unit in storage_units_list:
    storage_units[storage_unit] = MGridStorageUnit(sprice_list[storage_unit], sprice_probs[storage_unit], sprice_state[storage_unit], slife_list[storage_unit], slife_probs[storage_unit], slife_state[storage_unit], seff_list[storage_unit], seff_probs[storage_unit], seff_state[storage_unit], sdod_list[storage_unit], sdod_probs[storage_unit], sdod_state[storage_unit], sinitial_soc[storage_unit], slevels[storage_unit], sdecom_cost[storage_unit], sannual_om_cost_rate[storage_unit], sannual_degr_rate[storage_unit], scap[storage_unit], sloan_years[storage_unit], som_cost[storage_unit], srem_life[storage_unit], scur_eff[storage_unit], scur_dod[storage_unit])
num_facilities, num_power_plants, num_storage_units = len(facilities), len(power_plants), len(storage_units)
assert num_facilities == len(facilities_list), 'There are some facilities not entered in the dictionary!'
assert num_power_plants == len(power_plants_list), 'There are some power plants not entered in the dictionary!'
assert num_storage_units == len(storage_units_list), 'There are some storage units not entered in the dictionary!'

""" states definition """
start_state, states_map, states_scaler, cnt = [0], dict(), [decision_periods-1], 1
states_map['period'] = 0
external_features_power_plants, internal_features_power_plants = ['price', 'life', 'eff'], ['cap', 'rem_life', 'cur_eff']
external_features_storage_units, internal_features_storage_units = ['price', 'life', 'eff', 'dod'], ['cap', 'rem_life', 'cur_eff', 'cur_dod']
for power_plant in power_plants:
    for external_feature in external_features_power_plants:
        start_state.append(getattr(power_plants[power_plant], external_feature + '_list')[getattr(power_plants[power_plant], external_feature + '_state')])
        states_map[(power_plant, external_feature)] = cnt
        states_scaler.append(max(getattr(power_plants[power_plant], external_feature + '_list')))
        cnt += 1
    for internal_feature in internal_features_power_plants:
        start_state.append(getattr(power_plants[power_plant], internal_feature))
        states_map[(power_plant, internal_feature)] = cnt
        if internal_feature == 'cap':
            states_scaler.append(max(power_plants[power_plant].levels))
        elif internal_feature == 'rem_life':
            states_scaler.append(max(power_plants[power_plant].life_list))
        elif internal_feature == 'cur_eff':
            states_scaler.append(1)
        else:
            raise ValueError('No scaler for this feature!')
        cnt += 1
for storage_unit in storage_units:
    for external_feature in external_features_storage_units:
        start_state.append(getattr(storage_units[storage_unit], external_feature + '_list')[getattr(storage_units[storage_unit], external_feature + '_state')])
        states_map[(storage_unit, external_feature)] = cnt
        states_scaler.append(max(getattr(storage_units[storage_unit], external_feature + '_list')))
        cnt += 1
    for internal_feature in internal_features_storage_units:
        start_state.append(getattr(storage_units[storage_unit], internal_feature))
        states_map[(storage_unit, internal_feature)] = cnt
        if internal_feature == 'cap':
            states_scaler.append(max(storage_units[storage_unit].levels))
        elif internal_feature == 'rem_life':
            states_scaler.append(max(storage_units[storage_unit].life_list))
        elif internal_feature == 'cur_eff':
            states_scaler.append(1)
        elif internal_feature == 'cur_dod':
            states_scaler.append(1)
        else:
            raise ValueError('No scaler for this feature!')
        cnt += 1
start_state = np.array(start_state)
start_state = start_state.reshape(1,len(start_state))
states_dim = len(start_state[0])
def norm_state(s):
    return s / np.array(states_scaler)
# total_number_states = 
# example: state = [0, li_price, la_price, vr_price, fw_price, li_cap, la_cap, vr_cap, fw_cap]

""" actions definiton """
actions_map = [0]
for power_plant in power_plants:
    actions_map.append((power_plant, 0))
    for level in power_plants[power_plant].levels:
        actions_map.append((power_plant, level))
for storage_unit in storage_units:
    actions_map.append((storage_unit, 0))
    for level in storage_units[storage_unit].levels:
        actions_map.append((storage_unit, level))
actions_dim = len(actions_map)
actions = np.linspace(0, actions_dim-1, actions_dim)
actions = list(map(int, actions))
# total_number_states_actions = total_number_states * actions_dim
# actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# action_tuples = [0, (0,1), (0,2), (0,3), (1,1), (1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)]
# example: action = 5, action_tuple = (1,2)

""" reset the environment to the initial conditions """
def reset_environment():
    for power_plant in power_plants:
        power_plants[power_plant].price_state = 0
        power_plants[power_plant].life_state = 0
        power_plants[power_plant].eff_state = 0
        power_plants[power_plant].om_cost = pom_cost[power_plant]
        power_plants[power_plant].cap = pcap[power_plant]
        power_plants[power_plant].loan_payment = 0
        power_plants[power_plant].rem_loan = 0
        power_plants[power_plant].rem_life = prem_life[power_plant]
        power_plants[power_plant].cur_eff = pcur_eff[power_plant]
        if settings['random_transition_probabilities']:
            power_plants[power_plant].price_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
            power_plants[power_plant].life_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
            power_plants[power_plant].eff_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
    for storage_unit in storage_units:
        storage_units[storage_unit].price_state = 0
        storage_units[storage_unit].life_state = 0
        storage_units[storage_unit].eff_state = 0
        storage_units[storage_unit].dod_state = 0
        storage_units[storage_unit].om_cost = som_cost[storage_unit]
        storage_units[storage_unit].cap = scap[storage_unit]
        storage_units[storage_unit].loan_payment = 0
        storage_units[storage_unit].rem_loan = 0
        storage_units[storage_unit].rem_life = srem_life[storage_unit]
        storage_units[storage_unit].cur_eff = scur_eff[storage_unit]
        storage_units[storage_unit].cur_dod = scur_dod[storage_unit]
        if settings['random_transition_probabilities']:
            storage_units[storage_unit].price_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
            storage_units[storage_unit].life_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
            storage_units[storage_unit].eff_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]
            storage_units[storage_unit].dod_probs = (decision_periods - 1) * [random.uniform(0,1)] + [0]

In [5]:
def read_electricity_data(start_date=settings['start_date'], end_date=settings['end_date']):
    total_electricity_data = []
    for facility in facilities:
        dataset = pd.read_csv(settings['filename_electricity'] + facility + '.csv')
        electricity_data = dataset[['Date/Time', 'Electricity:Facility [kW](Hourly)']]
        electricity_data = pd.DataFrame.drop_duplicates(electricity_data)
                
        electricity_data['Electricity:Facility [kW](Hourly)'] = electricity_data['Electricity:Facility [kW](Hourly)'].replace(' ', '') # remove empty spaces
        electricity_data['Electricity:Facility [kW](Hourly)'] = electricity_data['Electricity:Facility [kW](Hourly)'].replace('Null', '-1') # handle missing data
        try:
            data_median = median(list(map(float,electricity_data['Electricity:Facility [kW](Hourly)'])))
            electricity_data['Electricity:Facility [kW](Hourly)'] = electricity_data['Electricity:Facility [kW](Hourly)'].replace('-1', str(data_median))
        except:
            pass
        
        new_series = pd.Series(data = electricity_data['Electricity:Facility [kW](Hourly)'].values, index = pd.date_range(start='2002-01-01 01:00:00', end='2003-01-01 00:00:00', freq='1h'))
        data_reindexed = electricity_data.reindex(pd.date_range(start=start_date, end=end_date, freq='1h'))  
        data_reindexed['Electricity:Facility [kW](Hourly)'] = new_series
        new_df = data_reindexed.reset_index()
        new_df = new_df.drop(columns=['Date/Time'])
        new_df = new_df.rename(columns={'index': 'Date/Time'})
        
        new_df = new_df.rename(columns={'Electricity:Facility [kW](Hourly)': 'Demand', 'Date/Time': 'Date_Time'})
        new_df = new_df[(new_df.Date_Time >= start_date) & (new_df.Date_Time <= end_date)]
        new_df['Demand'] = facilities[facility].cnt * new_df['Demand']
            
        new_df = new_df.reset_index()
        new_df = new_df.drop(columns=['index'])
        new_df.loc[:, 'Facility'] = facility
        if settings['include_ev_load']:
            ev_additional_load_daily = [facilities[facility].num_ev * facilities[facility].cnt * x for x in facilities[facility].ev_profile]
            ev_additional_load_total = ev_additional_load_daily * DAYS_IN_YEAR
            new_df['Demand'] += ev_additional_load_total[:-1]
        total_electricity_data.append(new_df)
    
    return total_electricity_data

def read_weather_data(start_date=settings['start_date'], end_date=settings['end_date']):
    dataset = pd.read_csv(settings['filename_weather'] + '.csv')
    weather_data = dataset[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'GHI', 'Wind Speed']]
    weather_data.loc[:, 'Year'] = '2002'
    
    weather_data = pd.DataFrame.drop_duplicates(weather_data)

    weather_data['Date_Time'] = pd.to_datetime(weather_data[['Year', 'Month', 'Day', 'Hour', 'Minute']])

    weather_data = weather_data.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'])
    weather_data = weather_data[(weather_data.Date_Time >= start_date) & (weather_data.Date_Time <= end_date)]
    
    """ handle missing values in weather dataframe """
    weather_data = weather_data.replace(-999,0)
    
    return weather_data

def interpolate_dataframe(df, frequency, min_datetime=settings['start_date'], max_datetime=settings['end_date']):
    df.index = df['Date_Time']
    df_reindexed = df.reindex(pd.date_range(start=min_datetime, end=max_datetime, freq=frequency))     
    df_interpolated = df_reindexed.interpolate(method='linear')
    new_df = df_interpolated.reset_index()
    new_df = new_df.drop(columns=['Date_Time'])
    new_df = new_df.rename(columns={'index': 'Date_Time'})
    
    return new_df.fillna(method='bfill')

def join_dataframes(df1, df2, based_on):  
    joint_df = pd.merge(df1, df2, how='inner', on=based_on)
    
    return joint_df.fillna(method='bfill')

""" read electricity and weather datasets """
electricity = read_electricity_data()
weather = read_weather_data()

""" interpolate and join dataframes """
w = interpolate_dataframe(weather, '30T')
e = []
for i, ele in enumerate(electricity):
    interpolated = interpolate_dataframe(electricity[i], '30T')
    if i == 0:
        interpolated = join_dataframes(interpolated, w, 'Date_Time')
    e.append(interpolated)
    
""" keep only 1-hour intervals """
delete_odd = [x for x in range(1,len(e[0]),2)]
single_df = []
for ele in e:
    ele = ele.drop(labels=delete_odd)
    single_df.append(ele)

""" create demand array """
for i, facility in enumerate(facilities):
    facilities[facility].yearly_data = single_df[i]['Demand'].values
for facility in facilities:
    facilities[facility].total_data = copy.deepcopy(facilities[facility].yearly_data)
for i in range(years_in_period):
    for facility in facilities:
        facilities[facility].total_data = np.append(facilities[facility].total_data, facilities[facility].yearly_data)
demand, demand_per_period = list(), list()
for i in range(decision_periods):
    to_append = list()
    for facility in facilities:
        to_append.append([(1 + settings['annual_demand_growth_rate_percent'] * years_in_period / 100) ** i * facilities[facility].crit_load * a for a in facilities[facility].total_data])
    demand_per_period.append(sum([np.mean(x) for x in to_append]))
    demand.append(to_append)

""" create weather array """
solar_ghi_orig, wind_speed_orig = single_df[0]['GHI'].values, single_df[0]['Wind Speed'].values
solar_ghi, wind_speed = [], []
meteo = dict()
for wp in settings['weather_predictors']:
    meteo[wp] = []
    for i in range(years_in_period+1):
        meteo[wp].extend(single_df[0][wp].values)
for i in range(years_in_period+1):
    solar_ghi.extend(solar_ghi_orig)
    wind_speed.extend(wind_speed_orig)
    
""" calculate solar production """
if 'solar' in power_plants_list:
    solar_cell_area, solar_cells_per_panel = 0.0232258, 72 # solar_area is the area of the solar cell
    # using solar_efficiency = 0.16, solar_cell_area = 0.0232258, solar_cells_per_panel = 72, then 20 solar panels output approximately 1kW
    power_plants['solar'].prod = [0.001 * solar_cell_area * solar_cells_per_panel * a for a in solar_ghi] # prod per solar panel without considering efficiency
    power_plants['solar'].power = np.mean(power_plants['solar'].prod)

""" calculate onshore wind production """
if 'onshore wind' in power_plants_list:
    wind_diameter, cut_in_speed, cut_out_speed = 44, 3, 22
    wind_area = math.pi * (wind_diameter / 2) ** 2 # wind_area is the rotor swept area
    # using wind_efficiency = 0.48, wind_diameter = 44, then 1 onshore wind turbine outputs approximately 37 kW
    power_plants['onshore wind'].prod = [0.001 * 0.5 * 1.25 * wind_area * a ** 3 if a >= cut_in_speed and a <= cut_out_speed else 0 for a in wind_speed]  # prod per onshore wind turbine without considering efficiency
    power_plants['onshore wind'].power = np.mean(power_plants['onshore wind'].prod)

""" calculate offshore wind production """
if 'offshore wind' in power_plants_list:
    wind_reduction = 0.2
    power_plants['offshore wind'].prod =  [0.001 * 0.5 * 1.25 * wind_area * (a * (1 / (1 - wind_reduction))) ** 3 if (a * (1 / (1 - wind_reduction))) >= cut_in_speed and (a * (1 / (1 - wind_reduction))) <= cut_out_speed else 0 for a in wind_speed]  # prod per offshore wind turbine without considering efficiency
    power_plants['offshore wind'].power = np.mean(power_plants['offshore wind'].prod)

""" calculate diesel production """
if 'diesel' in power_plants_list:
    power_plants['diesel'].prod = [1 for _ in solar_ghi]
    power_plants['diesel'].power = 0  # this basically means that diesel does not contribute to free production

""" calculate hydro production """
if 'hydro' in power_plants_list:
    power_plants['hydro'].prod = [1 for _ in solar_ghi]
    power_plants['hydro'].power = 0.5 # this is the capacity factor of hydropower

""" outage parameters """
if settings['superposed_scenario']: 
    saifi_extreme, saifi_normal, caidi_extreme, caidi_normal = 1 / 6, 0.84 / 6 + 0.89 / 6 + 0.76 / 6 + 1 / 6 + 1.34 / 6 + 1.1 / 6, 22.55, (1.65 + 1.42 + 1.95 + 1.46 + 1.7) / 5
else: 
    saifi_extreme, saifi_normal, caidi_extreme, caidi_normal = (1.84 + 0.89 + 0.76 + 1 + 1.34 + 1.1) / (6 * 2), (1.84 + 0.89 + 0.76 + 1 + 1.34 + 1.1) / (6 * 2), (22.55 + 1.65 + 1.42 + 1.95 + 1.46 + 1.7) / 6, (22.55 + 1.65 + 1.42 + 1.95 + 1.46 + 1.7) / 6
prob_extreme = saifi_extreme / (saifi_extreme + saifi_normal) # probability that an outage belongs to the extreme poisson process
prob_normal = 1 - prob_extreme
saifi_total = saifi_extreme + saifi_normal
scale_total_years = 1 / saifi_total
scale_total = scale_total_years * 8760 # scale parameter of the superposed poisson process converted in hours

In [6]:
""" model id and choose save/load """
nn_model_id = 1
nn_model_save = False
nn_model_load = False

""" set initial seed """
initial_seed = 13
np.random.seed(initial_seed)
random.seed(initial_seed)

""" nn and forecasting settings """
settings['architecture'] = 'ffnn'
settings['input_dim'] = 24
settings['output_dim'] = 1
settings['days_for_testing'] = 20
settings['split_ratio'] = (365 - settings['days_for_testing']) / 365
settings['use_forecasted_demand'] = False
settings['reserve_margin_rate'] = 0.05

""" nn hyperparameters """
nn_hparams = dict()
nn_hparams['activation'] = 'relu'
nn_hparams['num_layers'] = 1
nn_hparams['units'] = 32
nn_hparams['dropout'] = 0.2
nn_hparams['optimizer'] = 'Adam'
nn_hparams['learning_rate'] = 0.001
nn_hparams['beta_1'] = 0.9
nn_hparams['beta_2'] = 0.999
nn_hparams['loss'] = 'mse'
nn_hparams['epochs'] = 100
nn_hparams['early_stopping'] = True
nn_hparams['patience'] = int(0.03*nn_hparams['epochs'])
nn_hparams['filters'] = 2
nn_hparams['kernel_size'] = 2
nn_hparams['pool_size'] = 2

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def preprocess_data(x):
    scaler = MinMaxScaler()
    x = x.reshape(-1, 1)
    scaled_data = scaler.fit_transform(x)
    
    return scaled_data, scaler

def stack_multiple_series(elist, preprocessed=False, scaler=dict()):
    multiple_series = tuple()
    for i, facility in enumerate(facilities):
        sliced_df = elist[i]
        if i == 0:
            for wp in settings['weather_predictors']:
                arr = np.array(sliced_df[wp])
                if preprocessed:
                    arr = arr.reshape(-1, 1)
                    arr = scaler[wp].transform(arr)
                else:
                    arr, scaler[wp] = preprocess_data(arr)
                multiple_series += (arr.reshape((len(arr), 1)),)
            
        dem = np.array(sliced_df['Demand'])
        if preprocessed:
            dem = dem.reshape(-1, 1)
            dem = scaler['Demand_' + facility].transform(dem)
        else:
            dem, scaler['Demand_' + facility] = preprocess_data(dem)
        multiple_series += (dem.reshape((len(dem), 1)),)
    
    return np.hstack(multiple_series), scaler

def split_train_test(x, ratio):
    x_train = x[:int(ratio*len(x))]
    x_test = x[int(ratio*len(x)):]
    
    return x_train, x_test
    
def split_sequence(sequence, input_dim, output_dim, first_demand_column=0):
    """ split a univariate sequence into samples """
    x, y = list(), list()
    for i in range(input_dim,len(sequence),1):
        """ gather input and output parts of the pattern """
        seq_x, seq_y = sequence[i-input_dim:i], sequence[i:i+output_dim,first_demand_column:]
        if len(seq_y) < output_dim:
            break
        x.append(seq_x)
        y.append(seq_y)
        
    return np.array(x), np.array(y)

def model_create_ffnn(input_dim, output_dim, n_features, n_houses, x_train, y_train, x_test, y_test, early=None):
    """ model creation and training """
    model = Sequential()
    for _ in range(nn_hparams['num_layers']):
        model.add(Dense(nn_hparams['units'], activation=nn_hparams['activation'], input_shape=(input_dim,n_features)))
        model.add(Dropout(nn_hparams['dropout']))
    model.add(Flatten())
    model.add(Dense(y_train.shape[1]*y_train.shape[2]))
    custom_optimizer = getattr(optimizers, nn_hparams['optimizer'])(lr=nn_hparams['learning_rate'], beta_1=nn_hparams['beta_1'], beta_2=nn_hparams['beta_2'])
    model.compile(optimizer=custom_optimizer, loss=nn_hparams['loss'])
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]*y_train.shape[2]))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]*y_test.shape[2]))
    if early:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1, callbacks=[early])
    else:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1)
    model_loss = model.evaluate(x_train, y_train, verbose=0)
    
    return model, model_loss

def model_create_lstm(input_dim, output_dim, n_features, n_houses, x_train, y_train, x_test, y_test, early=None):
    """ model creation and training """
    model = Sequential()
    for _ in range(nn_hparams['num_layers']):
        model.add(LSTM(nn_hparams['units'], activation=nn_hparams['activation'], input_shape=(input_dim,n_features), return_sequences=True))
        model.add(Dropout(nn_hparams['dropout']))
    model.add(Flatten())
    model.add(Dense(y_train.shape[1]*y_train.shape[2]))
    custom_optimizer = getattr(optimizers, nn_hparams['optimizer'])(lr=nn_hparams['learning_rate'], beta_1=nn_hparams['beta_1'], beta_2=nn_hparams['beta_2'])
    model.compile(optimizer=custom_optimizer, loss=nn_hparams['loss'])
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]*y_train.shape[2]))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]*y_test.shape[2]))
    if early:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1, callbacks=[early])
    else:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1)
    model_loss = model.evaluate(x_train, y_train, verbose=0)
    
    return model, model_loss

def model_create_gru(input_dim, output_dim, n_features, n_houses, x_train, y_train, x_test, y_test, early=None):
    """ model creation and training """
    model = Sequential()
    for _ in range(nn_hparams['num_layers']):
        model.add(GRU(nn_hparams['units'], activation=nn_hparams['activation'], input_shape=(input_dim,n_features), return_sequences=True))
        model.add(Dropout(nn_hparams['dropout']))
    model.add(Flatten())
    model.add(Dense(y_train.shape[1]*y_train.shape[2]))
    custom_optimizer = getattr(optimizers, nn_hparams['optimizer'])(lr=nn_hparams['learning_rate'], beta_1=nn_hparams['beta_1'], beta_2=nn_hparams['beta_2'])
    model.compile(optimizer=custom_optimizer, loss=nn_hparams['loss'])
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]*y_train.shape[2]))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]*y_test.shape[2]))
    if early:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1, callbacks=[early])
    else:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1)
    model_loss = model.evaluate(x_train, y_train, verbose=0)
    
    return model, model_loss

def model_create_cnn(input_dim, output_dim, n_features, n_houses, x_train, y_train, x_test, y_test, early=None):
    """ model creation and training """
    model = Sequential()
    for _ in range(nn_hparams['num_layers']):
        model.add(Conv1D(filters=nn_hparams['filters'], kernel_size=nn_hparams['kernel_size'], activation=nn_hparams['activation'], input_shape=(input_dim,n_features)))
        model.add(MaxPooling1D(pool_size=nn_hparams['pool_size']))
    model.add(Flatten())
    model.add(Dropout(nn_hparams['dropout']))
    model.add(Dense(nn_hparams['units'], activation=nn_hparams['activation']))
    model.add(Dropout(nn_hparams['dropout']))
    model.add(Dense(nn_hparams['units'], activation=nn_hparams['activation']))
    model.add(Dropout(nn_hparams['dropout']))
    model.add(Dense(y_train.shape[1]*y_train.shape[2]))
    custom_optimizer = getattr(optimizers, nn_hparams['optimizer'])(lr=nn_hparams['learning_rate'], beta_1=nn_hparams['beta_1'], beta_2=nn_hparams['beta_2'])
    model.compile(optimizer=custom_optimizer, loss=nn_hparams['loss'])
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]*y_train.shape[2]))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]*y_test.shape[2]))
    if early:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1, callbacks=[early])
    else:
        model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=nn_hparams['epochs'], verbose=1)
    model_loss = model.evaluate(x_train, y_train, verbose=0)
    
    return model, model_loss

def model_predict(model, x_input):
    """ use the created model to get predictions for any unprocessed input """
    stacked, _ = stack_multiple_series(x_input, preprocessed=True, scaler=scales)
    stacked = stacked.reshape(-1,stacked.shape[0],stacked.shape[1])
    res = model.predict(stacked)
    res = res.reshape(-1, 1, 1)
    ans = []
    for i, facility in enumerate(facilities):
        res[i] = scales['Demand_' + facility].inverse_transform(res[i])
        ans.append(res[i][0][0])
    return ans
    
def performance_metrics(true, predicted, max_val, min_val, avg_val, choice):
    true = true.flatten()
    predicted = predicted.flatten()
    mse = mean_squared_error(true, predicted)
    rmse = sqrt(mse)
    if choice == 'divide_average':
      nrmse = rmse / avg_val
    elif choice == 'divide_range':
      nrmse = rmse / (max_val-min_val)
    errors = sorted(abs(true-predicted), reverse=True)
    errors = errors[:int(0.1*len(errors))]
    maxerr = sum(errors) / len(errors)
    if choice == 'divide_average':
      maxerr /= avg_val
    elif choice == 'divide_range':
      maxerr /= (max_val-min_val)
    
    return mse, rmse, nrmse, maxerr

def compute_performance_metrics():
    metrics, facility_profile = dict(), dict()
    metrics['mse_train'], metrics['rmse_train'], metrics['nrmse_train'], metrics['maxerr_train'] = 0, 0, 0, 0
    metrics['mse_test'], metrics['rmse_test'], metrics['nrmse_test'], metrics['maxerr_test'] = 0, 0, 0, 0
    for i, facility in enumerate(facilities):
        max_demand, min_demand, avg_demand = np.amax(np.concatenate((y_train[:,:,i], y_test[:,:,i]))), np.amin(np.concatenate((y_train[:,:,i], y_test[:,:,i]))), np.mean(np.concatenate((y_train[:,:,i], y_test[:,:,i])))
        facility_profile[facility] = dict()
        facility_profile[facility]['max_demand'] = max_demand
        facility_profile[facility]['min_demand'] = min_demand
        facility_profile[facility]['avg_demand'] = avg_demand
        metrics['mse_train_' + facility], metrics['rmse_train_' + facility], metrics['nrmse_train_' + facility], metrics['maxerr_train_' + facility] = performance_metrics(y_train[:,:,i], pred_train[:,:,i], max_demand, min_demand, avg_demand, 'divide_average')
        metrics['mse_test_' + facility], metrics['rmse_test_' + facility], metrics['nrmse_test_' + facility], metrics['maxerr_test_' + facility] = performance_metrics(y_test[:,:,i], pred_test[:,:,i], max_demand, min_demand, avg_demand, 'divide_average')
        metrics['mse_train'] += metrics['mse_train_' + facility]
        metrics['rmse_train'] += metrics['rmse_train_' + facility]
        metrics['nrmse_train'] += metrics['nrmse_train_' + facility]
        metrics['maxerr_train'] += metrics['maxerr_train_' + facility]
        metrics['mse_test'] += metrics['mse_test_' + facility]
        metrics['rmse_test'] += metrics['rmse_test_' + facility]
        metrics['nrmse_test'] += metrics['nrmse_test_' + facility]
        metrics['maxerr_test'] += metrics['maxerr_test_' + facility]  
    metrics['mse_train'] /= num_facilities
    metrics['rmse_train'] /= num_facilities
    metrics['nrmse_train'] /= num_facilities
    metrics['maxerr_train'] /= num_facilities
    metrics['mse_test'] /= num_facilities
    metrics['rmse_test'] /= num_facilities
    metrics['nrmse_test'] /= num_facilities
    metrics['maxerr_test'] /= num_facilities
    
    return metrics, facility_profile

def print_performance_metrics():
    print('\n---------- Facilities Profiles ----------')
    for facility in facilities:
        print('Maximum demand for the ', facility, ' facility is: ', profiles[facility]['max_demand'])
        print('Minimum demand for the ', facility, ' facility is: ', profiles[facility]['min_demand'])
        print('Average demand for the ', facility, ' facility is: ', profiles[facility]['avg_demand'])
    print('\n---------- Total Performance Metrics ----------')
    print('Total MSE (train): ', metrics['mse_train'])
    print('Total MSE (test): ', metrics['mse_test'])
    print('Total RMSE (train): ', metrics['rmse_train'])
    print('Total RMSE (test): ', metrics['rmse_test'])
    print('Total NRMSE (train): ', metrics['nrmse_train'])
    print('Total NRMSE (test): ', metrics['nrmse_test'])
    print('Total maxERR (train): ', metrics['maxerr_train'])
    print('Total maxERR (test): ', metrics['maxerr_test'])

""" stop execution if needed """
if nn_model_load:
    nn_model = load_model(file_path + '/nn_model_v' + str(nn_model_id) + '.h5')
    scales = pickle.load(open(file_path + '/scales.pkl', 'rb'))
    raise StopExecution
if not settings['use_forecasted_demand']:
    raise StopExecution
    
""" define existing function list for architectures and locations """
architecture_list = {'ffnn': model_create_ffnn, 'lstm': model_create_lstm, 'gru': model_create_gru, 'cnn': model_create_cnn}

""" stack multiple series """
serialized, scales = stack_multiple_series(single_df)

""" split to train and test set and then split sequenc
es  """
train_data, test_data = split_train_test(serialized, settings['split_ratio'])
x_train, y_train = split_sequence(train_data, settings['input_dim'], settings['output_dim'], weather.shape[1]-1)
x_test, y_test = split_sequence(test_data, settings['input_dim'], settings['output_dim'], weather.shape[1]-1)
    
""" create and fit the model """
n_features, n_facilities = x_train.shape[2], y_train.shape[2]
if nn_hparams['early_stopping']:
    es = EarlyStopping(monitor='val_loss', mode='min', patience=nn_hparams['patience'], verbose=1)
    nn_model, nn_model_loss = architecture_list[settings['architecture']](settings['input_dim'], settings['output_dim'], n_features, n_facilities, x_train, y_train, x_test, y_test, es)
else:
    nn_model, nn_model_loss = architecture_list[settings['architecture']](settings['input_dim'], settings['output_dim'], n_features, n_facilities, x_train, y_train, x_test, y_test)

""" make predictions for train and test set and inverse transform """
pred_train = nn_model.predict(x_train)
pred_test = nn_model.predict(x_test)
pred_train, pred_test = pred_train.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2])), pred_test.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2]))
for i, facility in enumerate(facilities):
    pred_train[:,:,i] = scales['Demand_' + facility].inverse_transform(pred_train[:,:,i])
    pred_test[:,:,i] = scales['Demand_' + facility].inverse_transform(pred_test[:,:,i])
    y_train[:,:,i] = scales['Demand_' + facility].inverse_transform(y_train[:,:,i])
    y_test[:,:,i] = scales['Demand_' + facility].inverse_transform(y_test[:,:,i])
    
""" compute performance metrics for train and test set """
metrics, profiles = compute_performance_metrics()

""" print performance metrics  for train and test """
print_performance_metrics()

""" save results if needed """
if nn_model_save:
    nn_model.save(file_path + '/nn_model_v' + str(nn_model_id) + '.h5') # save model
    pickle.dump(scales, open(file_path + '/scales.pkl', 'wb')) # save scalers
    nn_parameters = dict()
    nn_parameters['settings'], nn_parameters['nn_hparams'], nn_parameters['metrics'] = settings, nn_hparams, metrics
    with open(file_path + '/nn_parameters_v' + str(nn_model_id) + '.txt', 'w') as outfile: # save parameters
        json.dump(nn_parameters, outfile)

In [7]:
""" prepare input for neural network """
def prepare_nn_input(p, start):
    res = [dict() for _ in range(len(facilities))]
    for i, facility in enumerate(facilities):
        if i == 0:
            for wp in settings['weather_predictors']:
                res[i][wp] = meteo[wp][start-settings['input_dim']:start]
        res[i]['Demand'] = demand[p][i][start-settings['input_dim']:start]
    return res

""" value of lost load as a function of outage duration """
def compute_voll(a, b, x):
    return a + math.exp(b*x) if settings['use_exponential_voll'] else a
            
""" helper function """
def outage_simulation_helper(period, runs):
    
    """ outage generation """
    outage_time, outage_duration = list(), list()
    for i in range(runs):
        temp_time, temp_duration, time_sum = list(), list(), 0
        time_sum = round(np.random.exponential(scale_total))
        while time_sum < settings['input_dim']:
            time_sum = round(np.random.exponential(scale_total))
        #if time_sum > years_in_period * HRS_IN_YEAR: 
            #print('\nThe run number ' + str(i+1) + ' has 0 outages!\n')
        while time_sum <= years_in_period * HRS_IN_YEAR:
            temp_time.append(time_sum)
            if np.random.uniform(0, 1) < prob_extreme: 
                temp_duration.append(1 + np.random.poisson(caidi_extreme - 1))
            else:
                temp_duration.append(1 + np.random.poisson(caidi_normal - 1))
            time_sum += round(np.random.exponential(scale_total))
        outage_time.append(temp_time)
        outage_duration.append(temp_duration)
       
    """ tech initialization """
    tech_max, tech_dod_temp, tech_eff_temp = [], [], []
    for storage_unit in storage_units:
        tech_max.append(storage_units[storage_unit].cap)
        tech_dod_temp.append(storage_units[storage_unit].cur_dod)
        tech_eff_temp.append(storage_units[storage_unit].cur_eff)
    tech_min = [a * (1 - b) for a, b in zip(tech_max, tech_dod_temp)]
    tech_sum_discharge, tech_sum_charge = [a * b * c for a, b, c in zip(tech_max, tech_eff_temp, tech_dod_temp)], [d * (1 / e) * f  if e != 0 else 0 for d, e, f in zip(tech_max, tech_eff_temp, tech_dod_temp)] # compute the eff (or 1 / eff) * max * dod values for every technology
    sum_tech_sum_discharge, sum_tech_sum_charge = sum(tech_sum_discharge), sum(tech_sum_charge)
    if sum_tech_sum_discharge != 0 and sum_tech_sum_charge != 0:
        tech_rate_discharge, tech_rate_charge = [x / sum_tech_sum_discharge for x in tech_sum_discharge],  [y / sum_tech_sum_charge for y in tech_sum_charge] # compute the rate at which every technology should contribute to load discharge or charge
    array_opr_cost, array_out_cost, array_lolp, array_ccp = runs * [0], runs * [0], runs * [num_facilities * [0]], runs * [num_facilities * [0]] # the final arrays containing results for every simulation run
        
    """ loop over every simulation run """
    for run in range(runs):
        run_opr_cost, run_out_cost, run_lolp, run_ccp = 0, 0, num_facilities * [0], num_facilities * [0] # results for each simulation run
        free_total_production = 0
        for power_plant in power_plants:  # calculate operating costs for when there are no outages
            free_total_production += power_plants[power_plant].cur_eff * power_plants[power_plant].cap * power_plants[power_plant].power
        run_opr_cost = - min(free_total_production, demand_per_period[period]) * (HRS_IN_YEAR * years_in_period - sum(outage_duration[run])) * ELECTRICITY_PRICE_PER_KWH
        how_many_outages = len(outage_time[run])
        if how_many_outages == 0:
            array_lolp[run], array_ccp[run], array_opr_cost[run], array_out_cost[run] = num_facilities * [0], num_facilities * [1], run_opr_cost, 0
            continue
            
        """ loop over every outage in run """
        for timeout, duration in zip(outage_time[run], outage_duration[run]):
            hourly_lolp, hours_without_power = num_facilities * [0], num_facilities * [0]
            tech_energy = [storage_units[storage_unit].initial_soc * storage_units[storage_unit].cap for storage_unit in storage_units] # energy stored in each storage technology
            tech_avail_energy = [(x1 - x2) * x3 for x1, x2, x3 in zip(tech_energy, tech_min, tech_eff_temp)] # available energy in each storage technology
                
            """ loop over every hour in outage """
            for hour in range(duration):
                cum_demand = []
                for i in range(num_facilities):
                    cum_demand.append(demand[period][i][timeout+hour])
                cum_free_production, cum_pay_production = 0, 0
                for power_plant in power_plants:
                    if power_plants[power_plant].power != 0:
                        cum_free_production += power_plants[power_plant].prod[timeout+hour] * power_plants[power_plant].cur_eff * power_plants[power_plant].cap
                    else:
                        cum_pay_production += power_plants[power_plant].prod[timeout+hour] * power_plants[power_plant].cur_eff * power_plants[power_plant].cap
                sum_tech_avail_energy, sum_cum_demand = sum(tech_avail_energy), sum(cum_demand)
                """ get nn demand forecast and reduce production and available energy if necessary """
                if settings['use_forecasted_demand']:
                    x_input = prepare_nn_input(period, timeout+hour)
                    cum_forecasted_demand = model_predict(nn_model, x_input)
                    cum_forecasted_demand = [x * (1 + settings['reserve_margin_rate']) for x in cum_forecasted_demand]
                    sum_cum_forecasted_demand = sum(cum_forecasted_demand)
                    if cum_free_production + cum_pay_production + sum_tech_avail_energy > sum_cum_forecasted_demand:
                        difference = cum_free_production + cum_pay_production + sum_tech_avail_energy - sum_cum_forecasted_demand
                        rate_free_production, rate_pay_production, rate_avail_energy = cum_free_production / (cum_free_production + cum_pay_production + sum_tech_avail_energy), cum_pay_production / (cum_free_production + cum_pay_production + sum_tech_avail_energy), sum_tech_avail_energy / (cum_free_production + cum_pay_production + sum_tech_avail_energy)
                        cum_free_production -= difference * rate_free_production
                        cum_pay_production -= difference * rate_pay_production
                        sum_tech_avail_energy -= difference * rate_avail_energy
                if cum_free_production >= sum(cum_demand): # renewables production enough to satisfy the demand
                    energy_amount = cum_free_production - sum_cum_demand
                    if sum_tech_sum_discharge != 0 and sum_tech_sum_charge != 0:
                        tech_energy = [min(a + b * energy_amount * c, d) for a, b, c, d in zip(tech_energy, tech_rate_charge, tech_eff_temp, tech_max)]
                    tech_avail_energy = [(x1 - x2) * x3 for x1, x2, x3 in zip(tech_energy, tech_min, tech_eff_temp)]
                elif cum_free_production + cum_pay_production >= sum_cum_demand: # renewables and diesel production enough to satisfy the demand
                    run_opr_cost += (sum_cum_demand - cum_free_production) * DIESEL_PRICE_PER_KWH
                elif cum_free_production + cum_pay_production + sum_tech_avail_energy >= sum_cum_demand: # renewables and diesel production together with storage devices enough to satisfy the demand
                    run_opr_cost += cum_pay_production * DIESEL_PRICE_PER_KWH
                    energy_amount = sum_cum_demand - cum_free_production - cum_pay_production
                    if sum_tech_sum_discharge != 0 and sum_tech_sum_charge != 0:
                        tech_energy = [a - (b * energy_amount) / c if c != 0 else 0 for a, b, c in zip(tech_energy, tech_rate_discharge, tech_eff_temp)]
                    tech_avail_energy = [(x1 - x2) * x3 for x1, x2, x3 in zip(tech_energy, tech_min, tech_eff_temp)]
                else: # microgrid not able to satisfy the demand
                    jcount, temp_free_prod, temp_pay_prod = -1, cum_free_production, cum_pay_production
                    while temp_free_prod + temp_pay_prod + sum_tech_avail_energy < sum(cum_demand[:jcount]):
                        jcount -= 1
                        if jcount == -num_facilities: break
                    facilities_satisfied = num_facilities + jcount
                    for i, facility in enumerate(facilities):
                        if i < facilities_satisfied:
                            hours_without_power[i] = 0
                            if temp_free_prod >= demand[period][i][timeout+hour]:
                                temp_free_prod -= demand[period][i][timeout+hour]
                            elif temp_free_prod + temp_pay_prod >= demand[period][i][timeout+hour]:
                                temp_free_prod = 0
                                temp_pay_prod -= demand[period][i][timeout+hour] - temp_free_prod
                                run_opr_cost += (demand[period][i][timeout+hour] - temp_free_prod) * DIESEL_PRICE_PER_KWH
                            else:
                                energy_amount = demand[period][i][timeout+hour] - temp_free_prod - temp_pay_prod
                                run_opr_cost += temp_pay_prod * DIESEL_PRICE_PER_KWH
                                temp_free_prod, temp_pay_prod = 0, 0
                                if sum_tech_sum_discharge != 0 and sum_tech_sum_charge != 0:
                                    tech_energy = [a - (b * energy_amount) / c if c != 0 else 0 for a, b, c in zip(tech_energy, tech_rate_discharge, tech_eff_temp)]
                                tech_avail_energy = [(x1 - x2) * x3 for x1, x2, x3 in zip(tech_energy, tech_min, tech_eff_temp)]
                        else:
                            hours_without_power[i] += 1
                            hourly_lolp[i] += 1
                            run_out_cost += demand[period][i][timeout+hour] * compute_voll(facilities[facility].voll, settings['exponential_multiplier'], hours_without_power[i])
                    energy_amount = temp_free_prod
                    if sum_tech_sum_discharge != 0 and sum_tech_sum_charge != 0:
                        tech_energy = [min(a + b * energy_amount * c, d) for a, b, c, d in zip(tech_energy, tech_rate_charge, tech_eff_temp, tech_max)]
                    tech_avail_energy = [(x1 - x2) * x3 for x1, x2, x3 in zip(tech_energy, tech_min, tech_eff_temp)]
                    """ end of loop over every hour in outage """
                        
            temp_lolp = [k / duration for k in hourly_lolp]
            temp_ccp = [1 if temp_lolp[i] <= facilities[facility].lolp_constraint else 0 for i, facility in enumerate(facilities)]
            run_lolp = [a + b for a, b in zip(run_lolp, temp_lolp)]
            run_ccp = [a + b for a, b in zip(run_ccp, temp_ccp)]
            """ end of loop over every outage in run """
                
        array_lolp[run] = [a / how_many_outages for a in run_lolp]
        array_ccp[run] = [b / how_many_outages for b in run_ccp]
        array_opr_cost[run] = run_opr_cost
        array_out_cost[run] = run_out_cost
        """ end of loop over every simulation run """
        
    lolp = np.mean(array_lolp, axis = 0) # final lolp statistic for each system configuration
    ccp = np.mean(array_ccp, axis = 0) # final ccp statistic for each system configuration
    opr_cost = np.mean(array_opr_cost)
    out_cost = np.mean(array_out_cost) + sum([years_in_period * facilities[facility].kkt_multiplier if ccp[i] < facilities[facility].ccp_constraint else 0 for i, facility in enumerate(facilities)]) # final cost statistic for each system configuration
    
    return opr_cost, out_cost, lolp, ccp

In [ ]:
### """ model id and choose save/load """
rl_model_id = 16
rl_model_save = False
rl_model_load = False

""" set initial seed """
initial_seed = 13
np.random.seed(initial_seed)
random.seed(initial_seed)

""" rl hyperparameters """
rl_hparams = dict()
rl_hparams['algorithm'] = 'ddqn'
rl_hparams['num_episodes'] = 100_000
rl_hparams['print_every'] = rl_hparams['num_episodes'] // 100 if rl_hparams['num_episodes'] >= 100 else rl_hparams['num_episodes']
rl_hparams['save_every'] = rl_hparams['num_episodes'] // 10
rl_hparams['update_target'] = rl_hparams['num_episodes'] // 1_000 if rl_hparams['num_episodes'] >= 1_000 else 1
rl_hparams['decision_periods'] = decision_periods
rl_hparams['buffer_size'] = 20_000
rl_hparams['gamma'] = 0.9
rl_hparams['epsilon'] = 0.99
rl_hparams['epsilon_min'] = 0.01
rl_hparams['lrate'] = 0.01
rl_hparams['tau'] = 1
rl_hparams['nn_units'] = 128
rl_hparams['nn_layers'] = 2
rl_hparams['batch_size'] = 32
rl_hparams['dropout'] = None
rl_hparams['verbose_loop'] = False
rl_hparams['normalize_states'] = True



class StopExecution(Exception):
    def _render_traceback_(self):
        pass

""" create a class for the MGrid Agent """
class MGridAgent:
    
    """ initialize the agent """
    def __init__(self, tau= 1, clip= (-500., 500.)):
        self.buffer = deque(maxlen=rl_hparams['buffer_size'])
        self.epsilon, self.epsilon_step = rl_hparams['epsilon'], (rl_hparams['epsilon'] - rl_hparams['epsilon_min']) / rl_hparams['num_episodes']
        self.experiences_counter = 0
        self.main_model = self.create_model()
        self.target_model = self.create_model()
        self.tau = rl_hparams['tau']
        self.clip = clip
    
    """ helper function for creating the deep neural networks """    
    def create_model(self):
        model = Sequential()
        model.add(Dense(rl_hparams['nn_units'], kernel_initializer=initializers.glorot_uniform(seed=initial_seed), input_dim=states_dim, activation='relu'))
        for _ in range(rl_hparams['nn_layers']):
            model.add(Dense(rl_hparams['nn_units'], kernel_initializer=initializers.glorot_uniform(seed=initial_seed), activation='relu'))
            if rl_hparams['dropout']:
                model.add(Dropout(rl_hparams['dropout']))
        model.add(Dense(actions_dim))
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=rl_hparams['lrate']))
        return model
    #self.target_model = ActionStateModel(self.state_dim, self.action_dim)
    """ function for agent action """
  #  def take_action(self, state):
   #     q_val = self.main_model.predict(norm_state(state).reshape(1,states_dim))[0]
   #     if np.random.uniform(0, 1) < self.epsilon:
    #        exp_values= np.exp(np.clip(q_val/self.tau, self.clip[0], self.clip[1]))
    #        probs = math.fa(exp_values / np.sum(exp_values))
     #       return np.random.choice(actions, p=probs)   # explore
   #    if rl_hparams['normalize_states']:
     #       return np.argmax(self.main_model.predict(norm_state(state))[0]) # exploit
     #   else:
      #      return np.argmax(self.main_model.predict(state)[0]) # exploit
    
    def take_action(self, state):
        total_counts = rl_hparams['num_episodes']
        Q= np.zeros(20)
        tau= rl_hparams['tau']
        total= sum([math.exp(val/tau) for val in Q])
        probs = [math.exp(val/tau)/total for val in Q]
        threshold = random.random()
        cum_prob = 0.0
        for i in range(len(probs)):
            cum_prob += probs[i]
            if cum_prob >threshold:
                return random.choice(actions)
            if rl_hparams['normalize_states']:
                return np.argmax(self.main_model.predict(norm_state(state))[0]) # exploit
            else:
                return np.argmax(self.main_model.predict(state)[0]) # exploit
        #if random.uniform(0,1) < self.epsilon:
           # return random.choice(actions)   # explore
        #if rl_hparams['normalize_states']:
           # return np.argmax(self.main_model.predict(norm_state(state))[0]) # exploit
        #else:
           # return np.argmax(self.main_model.predict(state)[0]) # exploit
        
    """ compute next state and reward based on state transition rules and reward function """
    def compute_nextstate_reward(self, state, action, num_runs):
        """ define next state """
        action_tuple = actions_map[action]
        state = state[0]
        period = int(state[0])
        next_state = [period+1] # periods go up by 1
        """ change the capacity based on the selected action and compute any retirement costs """
        retirement_cost = 0
        if action_tuple != 0:
            if action_tuple[0] in power_plants.keys(): # decision to install/replace a power plant
                retirement_cost = power_plants[action_tuple[0]].decom_cost * power_plants[action_tuple[0]].cap # decommissioning costs
                retirement_cost += power_plants[action_tuple[0]].rem_loan # remaining loan payment 
                power_plants[action_tuple[0]].install_new(action_tuple[1])
            else: # decision to install/replace a storage unit
                retirement_cost = storage_units[action_tuple[0]].decom_cost * storage_units[action_tuple[0]].cap # decommissioning costs
                retirement_cost += storage_units[action_tuple[0]].rem_loan # remaining loan payment
                storage_units[action_tuple[0]].install_new(action_tuple[1])

        """ compute operation and outage cost """
        operation_cost, outage_cost, _, _ = outage_simulation_helper(period, num_runs)

        """ make other necessary changes to states and compute loan payments and om cost """
        investment_cost, om_cost = 0, 0
        for power_plant in power_plants: # decrease remaining life of power plants
            if power_plants[power_plant].cap > 0:
                investment_cost += min(power_plants[power_plant].rem_loan, power_plants[power_plant].loan_payment * years_in_period)
                power_plants[power_plant].rem_loan = max(power_plants[power_plant].rem_loan - power_plants[power_plant].loan_payment * years_in_period, 0)
                om_cost += power_plants[power_plant].om_cost
                if power_plants[power_plant].rem_life <= years_in_period:
                    retirement_cost += power_plants[power_plant].decom_cost * power_plants[power_plant].cap
            power_plants[power_plant].decrease_life()
            for external_feature in external_features_power_plants:
                power_plants[power_plant].state_transition(external_feature)    # markov chain transitions for external features
                next_state.append(getattr(power_plants[power_plant], external_feature + '_list')[getattr(power_plants[power_plant], external_feature + '_state')])
            for internal_feature in internal_features_power_plants:
                next_state.append(getattr(power_plants[power_plant], internal_feature))
        for storage_unit in storage_units: # decrease remaining life of storage units
            if storage_units[storage_unit].cap > 0:
                investment_cost += min(storage_units[storage_unit].rem_loan, storage_units[storage_unit].loan_payment * years_in_period)
                storage_units[storage_unit].rem_loan = max(storage_units[storage_unit].rem_loan - storage_units[storage_unit].loan_payment * years_in_period, 0)
                om_cost += storage_units[storage_unit].om_cost
                if storage_units[storage_unit].rem_life <= years_in_period:
                    retirement_cost += storage_units[storage_unit].decom_cost * storage_units[storage_unit].cap  
            storage_units[storage_unit].decrease_life()    
            for external_feature in external_features_storage_units:
                storage_units[storage_unit].state_transition(external_feature)  # markov chain transitions for external features
                next_state.append(getattr(storage_units[storage_unit], external_feature + '_list')[getattr(storage_units[storage_unit], external_feature + '_state')])
            for internal_feature in internal_features_storage_units:
                next_state.append(getattr(storage_units[storage_unit], internal_feature))            

        next_state = np.array(next_state)
        reward = - investment_cost - operation_cost - outage_cost - om_cost - retirement_cost
        return next_state.reshape(1,states_dim), reward
        
    """ update buffer """
    def buffer_update(self, state, action, reward, new_state, done):
        if rl_hparams['normalize_states']:
            state, new_state = norm_state(state).reshape(1,states_dim), norm_state(new_state).reshape(1,states_dim)
        else:
            state, new_state = state.reshape(1,states_dim), new_state.reshape(1,states_dim)
        self.buffer.append([state, action, reward, new_state, done])
        self.experiences_counter += 1
        
    """ experience replay for dqn algorithm """
    def experience_replay_dqn(self):
        if len(self.buffer) < rl_hparams['batch_size']:
            return
        x_batch, y_batch = [], []
        batch = random.sample(self.buffer, rl_hparams['batch_size'])
        
        for state, action, reward, new_state, done in batch:
            target = self.main_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                q_future = np.amax(self.target_model.predict(new_state)[0]) # select the best action and evaluate it on the target network
                target[0][action] = reward + q_future * rl_hparams['gamma']
            x_batch.append(state[0])
            y_batch.append(target[0])
        self.main_model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
        
    """ experience replay for ddqn algorithm """
    def experience_replay_ddqn(self):
        if len(self.buffer) < rl_hparams['batch_size']:
            return
        x_batch, y_batch = [], []
        batch = random.sample(self.buffer, rl_hparams['batch_size'])
        for state, action, reward, new_state, done in batch:
            target = self.main_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                best_action = np.argmax(self.main_model.predict(new_state)[0])  # select the best action on the main network
                q_future = self.target_model.predict(new_state)[0][best_action] # evaluate this action on the target network
                target[0][action] = reward + q_future * rl_hparams['gamma']
            x_batch.append(state[0])
            y_batch.append(target[0])
        self.main_model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
        
    """ experience replay for greedy algorithm """
    def experience_replay_greedy(self):
        if len(self.buffer) < rl_hparams['batch_size']:
            return
        x_batch, y_batch = [], []
        batch = random.sample(self.buffer, rl_hparams['batch_size'])
        for state, action, reward, new_state, done in batch:
            target = self.main_model.predict(state)
            target[0][action] = reward
            x_batch.append(state[0])
            y_batch.append(target[0])
        self.main_model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
        
    """ update the target network """
    def target_update(self):
        main_weights, target_weights = self.main_model.get_weights(), self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = main_weights[i] * rl_hparams['tau'] + target_weights[i] * (1 - rl_hparams['tau'])
        self.target_model.set_weights(target_weights)

    """ use the trained neural network to get q-values """
    def q_values(self, state, print_all=False):
        if rl_hparams['normalize_states']:
            q_val = self.main_model.predict(norm_state(state).reshape(1,states_dim))[0]
        else:
            q_val = self.main_model.predict(state.reshape(1,states_dim))[0]
        if print_all:
            print('\nOptimal action: ' + str(np.argmax(q_val)))
            print('\nQ-value for optimal action: ' + str(q_val[np.argmax(q_val)]))
            for i,v in enumerate(q_val):
                print('\nQ-value for action ' + str(i) + ': ' + str(v))
        return np.argmax(q_val), q_val
        
    """ save model """
    def save_model(self, snap):
        self.main_model.save(snap)
        
    def print_statistics(self, episode, treward):
        time.sleep(0)
        print('\n\nAverage cost for the percentile ' + str(episode//rl_hparams['print_every']) + ': ' + str(-treward//rl_hparams['print_every']))
        time_per_episode = (time.time() - start_time) / episode
        time_to_finish = time_per_episode * (rl_hparams['num_episodes'] - episode)
        time_to_finish_hours, time_to_finish = str(int(time_to_finish // SEC_IN_HOUR)), time_to_finish % SEC_IN_HOUR
        time_to_finish_minutes, time_to_finish = str(int(time_to_finish // SEC_IN_MIN)), time_to_finish % SEC_IN_MIN
        time_to_finish_seconds = str(int(time_to_finish // 1))
        if len(time_to_finish_hours) == 1:
            time_to_finish_hours = '0' + time_to_finish_hours
        if len(time_to_finish_minutes) == 1:
            time_to_finish_minutes = '0' + time_to_finish_minutes
        if len(time_to_finish_seconds) == 1:
            time_to_finish_seconds = '0' + time_to_finish_seconds
        print('\nApproximate time to finish: ' + str(time_to_finish_hours) + ':' + str(time_to_finish_minutes) + ':' + str(time_to_finish_seconds))
        time.sleep(0)
        return
    
    def print_final_statistics(self):
        return
    
    def print_figures(self,saved_rew):
        x_axis = [x for x in range(1,(rl_hparams['num_episodes']//rl_hparams['print_every'])+1)]
        plt.plot(x_axis,saved_rew)
        
""" agent initialization """
MGrid = MGridAgent()

""" load a pre-trained model """
if rl_model_load:
    MGrid.main_model = load_model(file_path + '/Saved Versions ' + str(rl_model_id) + '/MGrid_main_model_v' + str(rl_model_id) + '.h5.py')
    raise StopExecution

""" main function """
total_reward, saved_total_costs = 0, []
experience_replay = {'dqn': MGrid.experience_replay_dqn, 'ddqn': MGrid.experience_replay_ddqn, 'greedy': MGrid.experience_replay_greedy}
episode_iterator = [range(rl_hparams['num_episodes']), tqdm(range(rl_hparams['num_episodes']))]
start_time = time.time()
for episode in episode_iterator[rl_hparams['verbose_loop']]:
    reset_environment()
    MGrid.epsilon -= MGrid.epsilon_step
    cur_state = start_state
    print(episode)
    print(cur_state)
    """ save model instance """
    if rl_model_save and episode != 0 and episode % rl_hparams['save_every'] == 0:
        MGrid.main_model.save(file_path + '/MGrid_main_model_v' + str(rl_model_id) + '_' + str(episode // rl_hparams['save_every']) + '.h5')
    """ print statistics """
    if episode != 0 and episode % rl_hparams['print_every'] == 0:
        MGrid.print_statistics(episode,total_reward)
        saved_total_costs.append(-total_reward//rl_hparams['print_every'])
        total_reward = 0
    """ update target network """
    if episode != 0 and episode % rl_hparams['update_target'] == 0:
        MGrid.target_update()
    for decision_period in range(rl_hparams['decision_periods']):
        action = MGrid.take_action(cur_state)
        new_state, reward = MGrid.compute_nextstate_reward(cur_state, action, settings['num_outage_simulation_runs'])
        total_reward += reward
        done = (decision_period + 1 == rl_hparams['decision_periods'])
        MGrid.buffer_update(cur_state, action, reward, new_state, done)
        if MGrid.experiences_counter % rl_hparams['batch_size'] == 0:
            experience_replay[rl_hparams['algorithm']]()
        cur_state = new_state
MGrid.print_statistics(episode+1,total_reward)
saved_total_costs.append(-total_reward//rl_hparams['print_every'])
    
""" print final statistics """
MGrid.print_final_statistics()
MGrid.print_figures(saved_total_costs)

""" save results """
if rl_model_save:
    MGrid.main_model.save(file_path + '/MGrid_main_model_v' + str(rl_model_id) + '.h5.py') # save model
    parameters = dict()
    parameters['decision_periods'], parameters['years_in_period'], parameters['loan_horizon'], parameters['settings'], parameters['nn_hparams'], parameters['rl_hparams'] = decision_periods, years_in_period, loan_horizon, settings, nn_hparams, rl_hparams
    with open(file_path + '/MGrid_parameters_v' + str(rl_model_id) + '.txt', 'w') as outfile:
        json.dump(parameters, outfile)

In [ ]:
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click()
}setInterval(ClickConnect, 60000)